In [2]:
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm 
import json
import os
import datetime
import re
import glob
import csv
import time
import pandas as pd
from nltk.stem import *
from collections import Counter
from functools import reduce
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk import tokenize
#import datatable as dt   
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

#To read and store dictionary
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Matteo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [99]:
#TODO
#EDA
#do Lemmatization in the Data Cleaning 
#Create all the missing tsv
#Clean and comment the code
#CHECK EFFICIENCY 1.2 and 1.3
#Parse list before saving into
# finire di aggiungere labels al punto 4 (placeAdress,city)

### 1. Data collection
## 1.1. Get the list of places

In [ ]:
total_link = []
s = requests.Session()
for n in tqdm(range(1,401)):
    url = f'https://www.atlasobscura.com/places?page={n}&sort=likes_count'
    result = s.get(url)
    soup = bs(result.text)
    puf = soup.find_all("a", {'class': 'content-card content-card-place'})
    for x in puf:
        total_link.append(x['href'])


In [ ]:
f = open('total_link.txt',"r")
total_link = f.read().split("\n")

## 1.2. Crawl places

In [ ]:
def downloadPage(start,end, array):
    count_link = ((start-1)*18)+1
    count_page = start
    
    parent_dir = f'./all_Pages'
    s = requests.Session()
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    for x in tqdm(range(start, end)):
        if count_link%10 == 0:
            s = requests.Session()
        if count_link%18 == 1:
            path = os.path.join(parent_dir, f"folder_{count_page}")
            os.mkdir(path)
        for y in range(18):
            url = f'https://www.atlasobscura.com{array[count_link-1]}'
            name_file = f'location_{count_link}'
            name_folder = f'folder_{count_page}'
            with open(f'./all_Pages/{name_folder}/{name_file}.html', 'w', encoding='utf8') as fp:
                req = s.get(url, headers = header)
                fp.write(req.text)
                if req.status_code != 200:
                     time.sleep(120)
                     req = s.get(url, headers = header)
                fp.write(s.get(url).text)
            if count_link%18 == 0:
                count_page += 1
            count_link += 1
           
        

## 1.3 Parse downloaded pages

In [ ]:

def save_TSV():
    os.mkdir("TSV Files")
    for x in os.listdir("all_Pages"):
        for y in os.listdir(f'all_Pages/{x}'):
            with open(f'./all_Pages/{x}/{y}', encoding='utf8') as f:
                p = f.read()
                soup =  bs(p)
                pageAttribute = []
                placeName = findPlaceName(soup)
                placeTags = findPlaceTags(soup)
                numPeopleVisited = findNumPeopleVisited(soup)
                numPeopleWant = findNumPeopleWant(soup)
                placeDesc = findDescription(soup)
                placeShortDesc = findShortDescription(soup)
                placeNearby = findNearbyPlaces(soup)
                placeAddress = findAddress(soup)
                placeAlt, placeLong = findCordinates(soup)
                placeEditors = findPostEditors(soup)
                placePubDate = findPublishingDate(soup)
                placeRelatedList = findPlaceNear(soup)
                placeRelatedPlaces = findRelatedPlaces(soup)
                placeURL = findPageURL(soup)
                with open(f'./TSV Files/{y[:-5]}.tvs', 'wt', encoding='utf8') as fp:
                    csv.writer(fp, delimiter='\t').writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])
            

In [ ]:

def download_TSV():
    os.mkdir("TSV Files")
    a = 0
    for path in tqdm(glob.glob(r"all_Pages/*/*")):
        with open(path, encoding='utf8') as f:
                    a += 1
                    p = f.read()
                    soup =  bs(p)
                    pageAttribute = []
                    placeName = findPlaceName(soup)
                    placeTags = findPlaceTags(soup)
                    placeTags = ",".join(placeTags)
                    numPeopleVisited = findNumPeopleVisited(soup)
                    numPeopleWant = findNumPeopleWant(soup)
                    placeDesc = findDescription(soup)
                    placeDesc = " ".join(placeDesc)
                    placeShortDesc = findShortDescription(soup)
                    placeNearby = findNearbyPlaces(soup)
                    placeNearby = ",".join(placeNearby)
                    placeAddress = findAddress(soup)
                    placeAlt, placeLong = findCordinates(soup)
                    placeEditors = findPostEditors(soup)
                    placeEditors = ",".join(placeEditors)
                    placePubDate = findPublishingDate(soup)
                    placeRelatedList = findPlaceNear(soup)
                    placeRelatedList = ",".join(placeRelatedList)
                    placeRelatedPlaces = findRelatedPlaces(soup)
                    placeRelatedPlaces = ",".join(placeRelatedPlaces)
                    placeURL = findPageURL(soup)
                    with open(f'./TSV Files/{a}.tvs', 'wt', encoding='utf8') as fp:
                        csv.writer(fp, delimiter='\t').writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])
        
        

In [ ]:
download_TSV()

In [ ]:
def findPlaceName(soup):
    placeName = soup.find("h1", {"class": "DDPage__header-title"})
    if placeName != None:
        placeName = placeName.text
    #placeName = re.sub('[A-Za-z0-9_.,! "]*' ,'',placeName)
    return placeName

def findPlaceTags(soup):
    tags = []
    placeTags = soup.find_all("a", {"class": "itemTags__link js-item-tags-link"})
    for tag in placeTags:
        t = tag.text.replace("\n", "")
        #t = re.sub('[A-Za-z0-9 _.,!"]*','',t)
        tags.append(t)
    return tags

def findNumPeopleVisited(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[0].text)
    return peopleVisited

def findNumPeopleWant(soup):
    peopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})
    if len(peopleVisited) > 0:
        peopleVisited = int(peopleVisited[1].text)
    return peopleVisited

def findDescription(soup):
    all_description = []
    descriptions = soup.find_all("div", {"class": "DDP__body-copy"})
    for description in descriptions:
        d = description.text.replace("\n","")
        all_description.append(d)
    return all_description

def findShortDescription(soup):
    shortDescription = soup.find("h3", {"class": "DDPage__header-dek"})
    if shortDescription != None:
        shortDescription = shortDescription.text.replace("\n", "")
    return shortDescription

def findNearbyPlaces(soup):
    nearPlaces = []
    nearbyPlaces = soup.find_all("div", {"class": "DDPageSiderailRecirc__item-title"})
    if nearbyPlaces != None:
        for place in nearbyPlaces:
            p = place.text.replace("\n","")
            nearPlaces.append(p)
        #Convert the list to set, and then back again to list to remove repetition
    return set(nearPlaces)

def findAddress(soup):
    strings = []
    adress_strings = soup.find("address", {"class": "DDPageSiderail__address"})
    if adress_strings != None:
        adress_strings = adress_strings.find("div")
        for info in adress_strings:
            s = info.text.replace("\n", "")
            if s != "":
                strings.append(s)
        if len(strings) > 3:
            return " ".join(strings[:3])
    else:
        return " "

def findCordinates(soup):

    cordinates = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"})
    if cordinates != None:
        return cordinates.text.replace("\n", "").replace(" ","").split(",")
    return " ", " "

def findPostEditors(soup):
    all_editors =[]
    editors = soup.find_all("a", {"class":"DDPContributorsList__contributor"})         
    for person in editors:
        s = person.text.replace("\n", "")
        all_editors.append(s)
    return all_editors

def findPublishingDate(soup):
    #Pick the right info
    dateString = soup.find("div", {"class":"DDPContributor__name"}) 
    #Let's clean the string
    if dateString != None:
        s = dateString.text.replace("\n", "")
        #Let's modify it for the right format of datetime
        split = s.split()
        #Let's convert the string Month into the corrispondent number by using "strptime()" 
        split[0] = str(datetime.datetime.strptime(split[0], '%B').month)
        #My format
        format = "%m %d, %Y"
        #Convert from String to datetime
        date = datetime.datetime.strptime(" ".join(split), format)
        return date
    else:
        return ""

def findPlaceNear(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Nearby"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findRelatedPlaces(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Related"})
    if relatedLists != None:
        relatedLists = relatedLists.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
        for list in relatedLists:
            s = list.text.replace("\n", "")
            #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
            lists.append(s)
    return lists

def findPageURL(soup):
    numVisitedPeople = soup.find("link", {"rel":"canonical"})
    return numVisitedPeople['href']



## Load all the tsv file into one pandas dataframe

We decided to save all the .tsv file as one joined .csv where every file rappresents one row of the .csv file, so that we can work and access the data without having to read 7200 tsv file each time. The data are stored in the "tsv_dataframe"

This mean that the function load_tsv() has only been executed once. The other iteration we just read the .csv file

In [ ]:

def load_tsv() -> pd.DataFrame:
    tsv = []
    dtypes = {}
    for x in tqdm(os.listdir("TSV Files")):
        df = pd.read_csv(f'TSV Files/{x}',
            sep="\t",
            header=None,
            names=["placeName", "placeTags", "numPeopleVisited", "numPeopleWant", "placeDesc", "placeShortDesc", "placeNearby","placeAdress", "placeAlt", "placeLong", "placeEditors","placePubDate", "placeRelatedList", "placeRelatedPlace", "placeURL"])
        tsv.append(df)

    return pd.concat(tsv)

In [2]:
# data = load_tsv()
data = pd.read_csv("tsv_dataframe.csv",index_col=0)

#Reset Index
data.reset_index(inplace = True, drop=True)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   placeName          7200 non-null   object 
 1   placeTags          7166 non-null   object 
 2   numPeopleVisited   7200 non-null   int64  
 3   numPeopleWant      7200 non-null   int64  
 4   placeDesc          7200 non-null   object 
 5   placeShortDesc     7200 non-null   object 
 6   placeNearby        7200 non-null   object 
 7   placeAdress        7166 non-null   object 
 8   placeAlt           7200 non-null   float64
 9   placeLong          7200 non-null   float64
 10  placeEditors       7194 non-null   object 
 11  placePubDate       7199 non-null   object 
 12  placeRelatedList   7199 non-null   object 
 13  placeRelatedPlace  7166 non-null   object 
 14  placeURL           7200 non-null   object 
dtypes: float64(2), int64(2), object(11)
memory usage: 843.9+ KB


In [28]:
data.head()

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL
0,City Hall Station,"subways,subterranean,infrastructure,subterrane...",1834,8603,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"The Sugar House Prison Window,Tunnel Number 3,...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"Rebekah Otto,Annetta Black,Allan,charding407,w...",2010-05-08 00:00:00,"African Burial Ground National Monument,Tunnel...","Crystal Palace Subway,Moscow Metro Stations,Ro...",https://www.atlasobscura.com/places/city-hall-...
1,The Winchester Mystery House,"follies and grottoes,outsider architecture,ecc...",3723,5209,In 1886 an eccentric woman named Sarah Winches...,A peculiar mansion built by the troubled heir ...,"Rosicrucian Park,Rosicrucian Egyptian Museum,S...","525 South Winchester Boulevard San Jose, Calif...",37.3189,-121.9506,"mbison,Avoiding Regret,PrestonW,Collin,Destina...",2008-11-21 00:00:00,"Santana Row Chess Plaza,Rosicrucian Park,Rosic...","Casa de Piedra (Stone House),Portmeirion Villa...",https://www.atlasobscura.com/places/winchester...
2,Center for Puppetry Arts,"sesame street,puppets,museums",705,1027,"If you know your Gonzo from your Gelflings, yo...",The largest puppetry museum in the USA houses ...,"Breman Museum,The Consulate,Atlanta Monetary M...","1404 Spring St Atlanta, Georgia United States",33.7927,-84.3896,"kendallmbailey,Martin,andydoesit,We All Pod Do...",2018-11-12 00:00:00,"Breman Museum,Atlanta Monetary Museum,The Cons...","Portland Puppet Museum,Museu da Marioneta (Mus...",https://www.atlasobscura.com/places/center-for...
3,Amityville Horror House,"hoaxes and pseudoscience,haunted,film location...",367,831,"On a cold November night in 1974, Amityville, ...",The site of a tragic massacre that spawned a m...,"The Long Island Puppet Theater,Pilgrim Psychia...","112 Ocean Avenue Amityville, New York, 11701 U...",40.6665,-73.4143,"EricGrundhauser,MandrewPatinkin,lendog666,Mart...",2013-10-17 00:00:00,"The Long Island Puppet Theater,Fire Island Lig...","Conanicut Island Lighthouse,Lovell Health Hous...",https://www.atlasobscura.com/places/amityville...
4,Yuma Territorial Prison,"prisons,crime and punishment",422,833,"On July 1, 1876, the first seven inmates enter...",Walk through the actual strap iron cells and s...,"Valley of the Names,Felicity, California: Cent...","1 Prison Hill Rd Yuma, Arizona, 85364 United S...",32.7266,-114.6144,"desertfairy,mbison,CorpseLady84,Avoiding Regre...",2010-04-01 00:00:00,"Felicity, California: Center of the World,Vall...","Buckley's Cave,Palazzo d'Avalos,Oxford Castle ...",https://www.atlasobscura.com/places/yuma-terri...


In [3]:
data = data.fillna("")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   placeName          7200 non-null   object 
 1   placeTags          7200 non-null   object 
 2   numPeopleVisited   7200 non-null   int64  
 3   numPeopleWant      7200 non-null   int64  
 4   placeDesc          7200 non-null   object 
 5   placeShortDesc     7200 non-null   object 
 6   placeNearby        7200 non-null   object 
 7   placeAdress        7200 non-null   object 
 8   placeAlt           7200 non-null   float64
 9   placeLong          7200 non-null   float64
 10  placeEditors       7200 non-null   object 
 11  placePubDate       7200 non-null   object 
 12  placeRelatedList   7200 non-null   object 
 13  placeRelatedPlace  7200 non-null   object 
 14  placeURL           7200 non-null   object 
dtypes: float64(2), int64(2), object(11)
memory usage: 843.9+ KB


# 2. Search Engine

In [19]:
#TODO
#1. cambiare 'dic1' con 'vocabulary'
#2. cambiare 'dic2' con 'inverted_index'
#3. cambiare 'dic3' con 'inverted_index_2'
#4. calcolare nuovo dizionario 3

### funzioni poi da togliere dal file principale

In [6]:


def remove_punctuations(string) -> list:
    # first we remove the punctuations
    # in order to do it we need to tekenize the string with the function tokenize and then applying the function RegexpTokenizer
    return RegexpTokenizer(r'\w+').tokenize(string)


def stemming(string) -> list:
    # now we move forward with the stemming
    porter = PorterStemmer()
    string_stem=[porter.stem(word) for word in string]
    # we can now return the cleaned string 
    return string_stem

def remove_stopwords(string) -> list:
    # after this we can now remove all the stopwords in each word in string_t
    return  [word for word in string if not word.lower() in set(stopwords.words("english"))]
    # now we move forward with the stemming

def remove_numbers(string) -> list:
    for num in range(len(string)):
        if any(chr.isdigit() for chr in string[num]):
            string[num] = ''.join((x for x in string[num] if not x.isdigit()))
    return string



def cleaning(string) -> list:
    #I apply all the function for cleaning the string
    string = string.lower()
    string = remove_punctuations(string)
    string = remove_stopwords(string)
    string = remove_numbers(string)
    string = stemming(string)

    #return a list containing all the words of the original string after the cleaning
    return string

In [181]:
def save_dic(dic, name):
    with open(f'./Dictionary/{name}.pkl', 'wb') as f:
        pickle.dump(dic, f)

In [182]:
def read_dic(name):
    with open(f'./Dictionary/{name}.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
        return loaded_dict

Before starting the search engines, since we'll look for words in the descriptions of each place, we clean the collected placeDesc for every place in the dataset.  
In order to have later a more readable output we'll not sobsitute the column placeDesc with the one with all the cleaned descriptions but we'll add another column to our dataset 'data'.   
In particular the function *cleaning*, that we apply in the next cell at every description, will:
- convert all the upper characters in lower ones;
- remove all the punctuations;
- remove all the stopwords;
- remove all numbers;
- stem the string.

Let's create a new column with the clean descriptions:

In [ ]:
#The function cleaning takes as input a string (a description) and returns the cleaned description as a list of strings
data['cleanDesc'] = data.placeDesc.apply(lambda row: " ".join(cleaning(row)))

Now we save the data as a .csv file for future uses so that we don't need to run again the previous cell.

In [11]:
data.to_csv("dataframe_clean.csv")

Run the following cell if you want the original dataframe with a column 'cleanDesc' with the cleaned description.

In [45]:
data = pd.read_csv("dataframe_clean.csv")
data.fillna('', inplace=True)

## 2.1 Conjunctive query

### 2.1.1) Create your index!

The next function *createFirstDic*, given the dataset 'data', will return the dictionary *vocabulary* .     
Vocabulary will be such that every key is a unique word contained in the descriptions and its corresponing value is an integer that identifies that word.

In [4]:
def createFirstDic(data: pd.DataFrame, nameCol: str) -> dict:
    # first we create an empty dictionary
    dic = {}
    value = 1
    for description in tqdm(data[nameCol]):
        for word in description.split():
            # if the word is already in the dictionary keys this mean that it has already even a corresponding number and
            # so we can skip to the next word in the fixed description
            if word in dic.keys():
                continue
            # otherwise we create a new key named 'word' with value = value and then we upgrade value
            else:
                dic[word] = value
                value += 1
    return dic

We can now call the function createFirstDic and store its output as *vocabulary*:

In [42]:
dic1 = createFirstDic(data, "placeDesc")

100%|██████████| 7200/7200 [00:00<00:00, 19220.42it/s]


The second dictionary that we'll need for the first search engine is *inverted index* that is such that every key is an integer that identifies a word and its corresponding value is a set of all the document_id that contain that word.    
The following function *createSecondDic*, given as input the dataset 'data' and the dictionary 'vocabulary', will return the *inverted index* dictionary.  

In [5]:
def createSecondDic(data: pd.DataFrame, dic1: dict, nameCol: str) -> dict:
    dic2 = {}
    # a will keep track of the row index of the data
    a = 0
    # we take one description at a time from the column 'cleanDesc'
    for description in tqdm(data[nameCol]):
        # for every word in the fixed description 
        for word in description.split():
            # if the corresponding integer of the word is already in the keys of the inverted_index then we need to
            # add the document_id of the fixed description to the already existing set
            if dic1[word] in dic2.keys():
                dic2[dic1[word]].add(data.placeName[a])
            # otherwise we create a new key named as the corresponding integer of the word with value equal to a set that 
            # contains the document_id of the fixed description
            else:
                dic2[dic1[word]] = set([data.placeName[a]])
        a += 1
    return dic2

    

We can now call the function createSecondDic and store its output as *inverted_index*:

In [46]:
dic2 = createSecondDic(data,dic1, "placeDesc")

100%|██████████| 7200/7200 [00:14<00:00, 495.63it/s]


Even in this case we can store the two dictionary in a separate file so that we don't need to run the codes everytime we need them.

In [ ]:
save_dic(dic1, dic1)
save_dic(dic2,dic2)

Run the following cell if you want the dictionaries *vocabulary* and *inverted_index*.

In [7]:
dic1 = read_dic("dic1")
dic2 = read_dic("dic2")

### 2.1.2) Execute the query

First of all we ask the user which query do we have to look for in the places descriptions and we store it as a string named *q*:

In [8]:
q = input('Insert the query that you want to find in the places descriptions: ')

Insert the query that you want to find in the places descriptions: American Museum


Before starting the first search engine with the query *q* we need to clean q too.

In [9]:
q = cleaning(q) 
# now q is a list that contains the cleaned word of the input query
print(q)

['american', 'museum']


The next function *query* is our first search engine that, given as input a list of words (the query), it will return as output a list of all the documents_id that contain all the words in the list.

In [7]:
def query(list, dic1, dic2):
    # s is the set of the documents_id that contain the first word in the query
    s = dic2[dic1[list[0]]]
    # starting from the second word in the query till the last we'll intersect the set s with the set of all the documents_id 
    # that contain the fixed word 
    for x in range(1, len(list)):
        s.intersection(dic2[dic1[list[x]]])
    return s

We can now execute the previous function with our query q:

In [11]:
s = query(q)
# s is the set with all the documents_id of all the descriptions that contain all the words in the query

We can now create a new dataset *res_query* with just the places we're interested in:

In [12]:
res_query = data[data['placeName'].isin(list(s))]

With the following code we'll show the result of the first search engine.    
In particular we'll show just the places names, their description and their URL.

In [13]:
res_query[['placeName','placeDesc','placeURL']]

,placeName,placeDesc,placeURL
3,Amityville Horror House,"On a cold November night in 1974, Amityville, ...",https://www.atlasobscura.com/places/amityville...
6,American Prohibition Museum,When the 18th Amendment to the U.S. Constituti...,https://www.atlasobscura.com/places/american-p...
13,Wild Blueberry Land,There is no shortage of bizarre American highw...,https://www.atlasobscura.com/places/wild-blueb...
39,The Avrocar,The year was 1952 and the Cold War was in full...,https://www.atlasobscura.com/places/avrocar
59,Laurel Dinosaur Park,"During the 18th and 19th centuries, clay forma...",https://www.atlasobscura.com/places/laurel-din...
...,...,...,...
7155,Garden of Eden Trail,In the 1950s retired lawyer and Republican can...,https://www.atlasobscura.com/places/garden-of-...
7183,Theodore Roosevelt Birthplace Museum,Behind an otherwise innocuous (if immaculately...,https://www.atlasobscura.com/places/theodore-r...
7184,Cabaret Mechanical Theatre,"It’s not a theater and it’s not a cabaret, so ...",https://www.atlasobscura.com/places/cabaret-me...
7186,Timothy Demonbreun's Cave,Seemingly little more than a wide crack in the...,https://www.atlasobscura.com/places/timothy-de...


## 2.2) Conjunctive query & Ranking score

### 2.2.1) Inverted index

Now we have to create another dictionary *inverted_index_2* in which we store also the *tfidf* values of the words.   
As suggested, since the tfidf are invariant for the query we can precalculate them before and store them in a dataframe *df*.    
In order to do this we use the module *TfidfVectorizer* that is able to compute the tfidf values of all the unique words in the places descriptions.

In [14]:
vectorizer = TfidfVectorizer(use_idf=True, analyzer='word',sublinear_tf=True)
x =vectorizer.fit_transform(data["cleanDesc"]).todense()
df = pd.DataFrame(x, columns = vectorizer.get_feature_names_out())

*df* now is a dataframe such that every row corresponds to a place's description and every column corresponds to a word found in the descriptions.    
Using now *df* and the two dictionary previously created, we can now create *inverted_index_2* that will be such that every key is an integer that corresponds to a word and its value is a list of touples (document_id, tfidf{word,document_id}).

In [8]:
def createInvertedIndex(data, dic1, nameCol, matrix):
    inverted_index2 = {}
    # for every word in columns name of df
    a = 0
    for word in tqdm(matrix.columns):
        # we fix a row in df that correspond to a place's description
        for place in matrix.index:
            # if in the keys of our dictionary there already is the integer that corresponds to our word then we just have to 
            # add a tuple to our existing set with the placeName and the tfidf score of that word in the fixed description
            if dic1[word] in inverted_index2.keys() and matrix[word][place] != 0:
                inverted_index2[dic1[word]].add((data[nameCol][place], matrix[word][place]))
            # otherwise we can add a new key with value a set that contains a tuple (placeName,tfidf{word,placeName})
            else:
                inverted_index2[dic1[word]] = set((data[nameCol][place], matrix[word][place]))           
    return inverted_index2

We can now execute the previous function with as input our *data*, *vocabulary* and *df*:

In [17]:
dic3 = createInvertedIndex(data, dic1, df)

100%|██████████████████████████████████████████████████████████████████████████████| 7200/7200 [12:30<00:00,  9.59it/s]


As previously done with the other two dictionary, we store *inverted_index_2* in a separate file so that we don't have to compute it again.

In [20]:
save_dic(dic3, "dic3")

Run the following cell if you want the dictionariy *inverted_index_2*.

In [8]:
dic3 = read_dic("dic3")

Have a look at our dictionary:

### 2.2.2) Execute the query

In [33]:
def query_tfidf(data: pd.DataFrame, nameCol: str, q: str) -> pd.DataFrame:
    #We need to clean the query
    q = cleaning(q) 

    # now q is a list that contains the cleaned word of the input query
    vectorizer = TfidfVectorizer(use_idf=True, analyzer='word',sublinear_tf=True)
    x =vectorizer.fit_transform(data["cleanDesc"]).todense()
    df = pd.DataFrame(x, columns = vectorizer.get_feature_names_out())
    qv = vectorizer.transform([" ".join(q)]).todense()
    
    # since later we'll need to compute the norm of this vector, we create an np array with the tfidf of the query
    qv = np.array(qv[0,:])

    dic1 = createFirstDic(data, nameCol)
    dic2 = createSecondDic(data, dic1, nameCol)
    s = query(q,dic1,dic2)
    
    # res_query will be our new dataframe such that contains only the places with description that 
    # contains all the words in the query
    res_query = data[data['placeName'].isin(list(s))]

    scores = cosine_score(res_query,df,qv)
    res_query.insert(res_query.shape[1], f"CS_{nameCol}",scores )   

    res_query = res_query.sort_values(by=[f"CS_{nameCol}"],ascending=False)

    return res_query


In [34]:
q = input('Insert the query that you want to find in the places descriptions: ')
query_tfidf(data, "cleanDesc", q).head(10)

100%|██████████| 7200/7200 [00:23<00:00, 312.16it/s]


,Unnamed: 0,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL,cleanDesc,CS_cleanDesc
4563,4563,Self-Taught Genius Gallery,"art museum,outsider art,art",51,457,"In 2017, the American Folk Art Museum in Manha...",An exhibition space dedicated to American outs...,"Surreal Elevator ,Smiling Hogshead Ranch,Kings...","32-23 48th Ave Queens, New York, 11101 United ...",40.7409,-73.9334,"Rachel Gould,Anna Minster,reneesoto",2019-08-21 00:00:00,"Surreal Elevator ,Smiling Hogshead Ranch,Kings...","Ilana Goor Museum ,St Catherine's Passage,Bob ...",https://www.atlasobscura.com/places/self-taugh...,american folk art museum manhattan open offsh...,0.181039
605,605,Renwick Gallery,"art,museums,architecture",1272,754,The Renwick Gallery building has the distincti...,The first purpose-built art gallery in the Uni...,"D.C.'s Underground Bald Cypress Fossils,FDR's ...","Washington, District of Columbia, 20506 United...",38.8990,-77.0391,"lionsinwanderland,hrnick,paulypaul,gowens,vcoll82",2015-12-15 00:00:00,"D.C.'s Underground Bald Cypress Fossils,FDR's ...","Blanco Renaissance Museum ,Espai Xavier Corber...",https://www.atlasobscura.com/places/renwick-ga...,renwick galleri build distinct first art museu...,0.180539
2216,2216,American Writers Museum,writers,182,599,The American Writers Museum—tucked away on the...,"Small and slightly hidden, the American Writer...","Tiffany Dome,Heald Square Monument's Statue of...","180 N Michigan Ave Chicago, Illinois, 60601 Un...",41.8856,-87.6249,clairers,2019-06-28 00:00:00,"Tiffany Dome,Heald Square Monument's Statue of...","The Writers Chair,Casa Fernando Pessoa,Grave o...",https://www.atlasobscura.com/places/american-w...,american writer museum tuck away second floor ...,0.176558
2585,2585,Mount Angel Abbey Museum,"wunderkammers,natural history museums,reposito...",268,572,The Mount Angel Abbey Museum is a very eclecti...,World's largest porcine hairball sits in a col...,"The Gordon House ,Wooden Shoe Tulip Farm,TMK C...","One Abbey Drive Mount Angel, Oregon, 97373 Uni...",45.0553,-122.7710,"fatpirate,darkdaybreak,Nicholas Jackson",2011-02-09 00:00:00,"The Gordon House ,Wooden Shoe Tulip Farm,TMK C...","Tullaherin Folk Museum and Round Tower,Palais ...",https://www.atlasobscura.com/places/mount-ange...,mount angel abbey museum eclect collect put to...,0.168144
4413,4413,The American Pigeon Museum,"pigeons,birds,animals",137,464,"Many people, especially in big cities, view pi...",Celebrate the history of the rich and truly mi...,"Oklahoma State Firefighters Museum ,Rodeo Anim...","2300 NE 63rd St Oklahoma City, Oklahoma, 73111...",35.5361,-97.4703,"EricGrundhauser,cait7911,Collector of Experiences",2015-06-30 00:00:00,"Rodeo Animal Cemetery,National Cowboy and West...","Warrior Birds Memorial,Paddy the Pigeon Plaque...",https://www.atlasobscura.com/places/the-americ...,mani peopl especi big citi view pigeon fli rat...,0.167326
102,102,Museum of Mourning Art,"funeral cars,funeral art,cultures and civiliza...",53,820,Mourning and personal response to death are un...,Arlington Cemetery exhibit dedicated to death ...,"The Lower Swedish Cabin,Mount Moriah Cemetery,...","2900 State Road Upper Darby, Pennsylvania, 190...",39.9559,-75.2971,"atimian,MontyPySpock",2011-12-27 00:00:00,"The Lower Swedish Cabin,Unmarked Grave of H.H....","Museu de Carrosses Fúnebres de Barcelona,Museu...",https://www.atlasobscura.com/places/museum-of-...,mourn person respons death uniqu everi person ...,0.166433
5208,5208,Indian Steps Museum,,120,424,"Constructed by a local lawyer from 1908-1912, ...",A small museum along the Susquehanna River is ...,"'Newspaper Reader',Lancaster Crematorium,Peach...","205 Indian Steps Rd Airville, Pennsylvania Uni...",39.8634,-76.3755,"SpookyLittl3Girl,lemieuxruibal",2018-10-05 00:00:00,"Peach Bottom Slate,Lancaster Crematorium,'News...",,https://www.atlasobscura

First of all we ask the user which query do we have to look for in the places descriptions and we store it as a string named *q*:

In [15]:
q = input('Insert the query that you want to find in the places descriptions: ')

Insert the query that you want to find in the places descriptions: American Museum


Before starting the first search engine with the query *q* we need to clean q too.

In [16]:
q = cleaning(q) 
# now q is a list that contains the cleaned word of the input query
print(q)

['american', 'museum']


In order to compute the *cosine similarity* between the query and the descriptions we need to compute the *tfidf* of the words in the query too and we'll store them the in the variable *qv*:

In [17]:
qv = vectorizer.transform([" ".join(q)]).todense()
# since later we'll need to compute the norm of this vector, we create an np array with the tfidf of the query
qv = np.array(qv[0,:])

Before to compute the cosine similarity we have to slect all the places descriptions that contain all the words in the query.  
In order to do this we execute the function of the point 2.1.2 with as input our query *q*.

In [18]:
s = query(q)
# res_query will be our new dataframe such that contains only the places with description that 
# contains all the words in the query
res_query = data[data['placeName'].isin(list(s))]

The next function *cosine_score* given *res_query*, *df* and *qv* is able to compute the cosine similarity of the places descriptions with respect to the query.    
Let's remark what is the cosine similarity of a description to a query:
$$cosine similarity = \frac{sentence\cdot query}{norm(sentence)*norm(query)}$$
where *sentence* is the tfidf vector of the given sentence and *query* is the tfidf of the query.

In [10]:
def cosine_score(res_query,df,qv):
    # first we create an empty list in which store all the cosine similarity scores
    scores = []
    # for every place in res_query we compute the cosine similarity as shown before and append that value in the list scores
    for place in res_query.index:
        scores.append(np.dot(np.array(df.iloc[place]).reshape(1,df.shape[1])[0],qv[0]) / (np.linalg.norm(np.array(df.iloc[place]).reshape(1,df.shape[1])[0])*np.linalg.norm(qv[0]) ))
    return scores

We can compute the cosine similarity calling the previous function and store the list in a variable called *scores*.    
Then we add this list in a new column called 'cosine similarity'.

In [20]:
scores = cosine_score(res_query,df,qv)
res_query.insert(res_query.shape[1], "cosine similarity",scores )

Now we can sort the dataframe *res_query* by the column cosine similarity:

In [21]:
res_query = res_query.sort_values(by=['cosine similarity'],ascending=False)

We can now have a look at the final output:

In [25]:
res_query[['placeName','placeDesc','placeURL','cosine similarity']].head(10)

,placeName,placeDesc,placeURL,cosine similarity
4563,Self-Taught Genius Gallery,"In 2017, the American Folk Art Museum in Manha...",https://www.atlasobscura.com/places/self-taugh...,0.181039
605,Renwick Gallery,The Renwick Gallery building has the distincti...,https://www.atlasobscura.com/places/renwick-ga...,0.180539
2216,American Writers Museum,The American Writers Museum—tucked away on the...,https://www.atlasobscura.com/places/american-w...,0.176558
2585,Mount Angel Abbey Museum,The Mount Angel Abbey Museum is a very eclecti...,https://www.atlasobscura.com/places/mount-ange...,0.168144
4413,The American Pigeon Museum,"Many people, especially in big cities, view pi...",https://www.atlasobscura.com/places/the-americ...,0.167326
102,Museum of Mourning Art,Mourning and personal response to death are un...,https://www.atlasobscura.com/places/museum-of-...,0.166433
5208,Indian Steps Museum,"Constructed by a local lawyer from 1908-1912, ...",https://www.atlasobscura.com/places/indian-ste...,0.161671
3196,The American Kennel Club Museum of the Dog,At the intersection of the Venn diagram where ...,https://www.atlasobscura.com/places/the-americ...,0.161211
515,Museum of Russian Culture,"When you think San Francisco, you probably don...",https://www.atlasobscura.com/places/museum-of-...,0.160452
6185,Harvard Museum of Natural History,Collecting three different institutions into o...,https://www.atlasobscura.com/places/harvard-mu...,0.159486


## 3. Define a new score!

Our idea is to create a new score based on the number of people who have already been in a place or on the number of people who want to see that place. 
First we let the user insert the query that he want to search in the places descriptions.   
Then we let the user choose the number of places he wants to see in the final output and if he wants to sort the places by the scores of *numPeopleVisited* or of *numPeopleWant*.  
As last input we ask the user if he wants to see the most popular ones or the least.

## FINO A QUI

In [8]:
query_user = list(input("Insert the words that you want to be found in the description's places: ").split())

Insert the words that you want to be found in the description's places: church mountain


In [9]:
#first of all we execute the search engine as in 2.1 with the query given by the user
dic1 = read_dic("dic1")
dic2 = read_dic("dic2")
s = query(query_user)
res_query = data[data['placeName'].isin(list(s))]
res_query.reset_index(inplace = True, drop=True) 

In [10]:
n = int(input('What is the top rank visualization that you want to have? \nInsert "1" if you prefer that is by number of tourists. \nInsert "2" if you prefer that is by number of people that want to visit the site.\n'))

What is the top rank visualization that you want to have? 
Insert "1" if you prefer that is by number of tourists. 
Insert "2" if you prefer that is by number of people that want to visit the site.
2


In [11]:
k = int(input('Insert k the number of places that you want to visualize:')) 
m = int(input('Insert "1" if you want to see the k most popular places or insert "2" if you want to see the less famous places:'))

Insert k the number of places that you want to visualize:10
Insert "1" if you want to see the k most popular places or insert "2" if you want to see the less famous places:1


In [12]:
def new_score(res_query,n,k,m):
    scores = []
    if n == 1:
        by = res_query['numPeopleVisited']
    else:
        by = res_query['numPeopleWant']
    max_value = by.max()
    for i in range(res_query.shape[0]):
        scores.append(round(by[i]/max_value,4))
    res_query.insert(res_query.shape[1], "scores", scores)
    if m == 1:
        res_query = res_query.sort_values(by=['scores'],ascending=False)
    else:
        res_query = res_query.sort_values(by=['scores'],ascending=True)
        
    return res_query.head(k)

In [13]:
resquery = new_score(res_query,n,k,m)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL,scores
93,Dream House,"optical oddities,eccentric homes,music,outside...",765,6007,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","Aretha Franklin Subway Tributes,Farm.One,The '...","275 Church St New York, New York, 10013 United...",40.7185,-74.0048,"seanmattison,mbison,Gray,Sofy,erosika,Blindcol...",2009-09-22 00:00:00,"Aretha Franklin Subway Tributes,Farm.One,The '...","Callejon de Hamel,Casa Neverlandia,Casa de Azu...",https://www.atlasobscura.com/places/dream-house,1.0000
541,The Ruins of St. Dunstan-in-the-East,"gardens,world war ii,churches,plants,military,...",1884,5754,The church of St.Dunstan-in-the-East has survi...,One of the few remaining casualties of the Lon...,"Billingsgate Roman House and Baths,Pudding Lan...",St. Dunstan-in-the-East St. Dunstan's Hill Lon...,51.5097,-0.0827,"Luke J Spencer,Goksu,Kas,adamjerodandrews,Mons...",2015-03-12 00:00:00,"Billingsgate Roman House and Baths,Philpot Lan...","Kaiser Wilhelm Memorial Church,Ruins of Holyro...",https://www.atlasobscura.com/places/the-ruins-...,0.9579
92,Roosevelt Island Smallpox Hospital Ruins,"smallpox,quarantine,abandoned hospitals,abando...",1709,5153,Few diseases have had a greater impact on the ...,A crumbling hospital from the 1850s on Rooseve...,"United Nations Delegates Lounge,Roosevelt Isla...","Roosevelt Island New York, New York, 10044 Uni...",40.7515,-73.9596,"Rebekah Otto,Sharon Blackstone,spavalice,mbiso...",2010-01-13 00:00:00,"Roosevelt Island Cat Sanctuary,Strecker Memori...","Ruins of Lazaretto de Isla de Cabras,Glenn Dal...",https://www.atlasobscura.com/places/roosevelt-...,0.8578
262,The Hardy Tree,"ao loves halloween 2022,gravestones,trees,chur...",859,4042,"Inside an ancient London churchyard, an ash tr...",This churchyard arbor is surrounded by hundred...,"Platform 9 3/4,Word on the Water,British Libra...","Pancras Road London, England, NW1 1UL United K...",51.5349,-0.1309,"sarahvjanet,Pengwn,SEANETTA,giloscope,Stevren,...",2015-07-03 00:00:00,"Word on the Water,Platform 9 3/4,British Libra...",Convento de la Santa Cruz (Convent of Santa Cr...,https://www.atlasobscura.com/places/the-hardy-...,0.6729
481,Santa Maria della Concezione Crypts,"capuchins,marquis de sade,mark twain,ossuaries...",2070,3529,"In 1775, the Marquis de Sade wrote of it, “I h...",The crypts of Capuchin friars decorated with t...,"Saint Victoria's Incorruptible Body,Keats-Shel...","Via Vittorio Veneto, 27 Rome, 00187 Italy",41.9047,12.4882,"Dylan,mshkrvn,Annetta Black,The Dubious Hausfr...",2009-08-09 00:00:00,"Keats-Shelley Memorial House,Zuccari Palace,Sa...","The Czech's Capuchin Crypt,Santa Maria dell'Or...",https://www.atlasobscura.com/places/santa-mari...,0.5875
501,International Church of Cannabis,"cannabis,marijuana,drugs,religion,churches,sac...",607,3206,A Lutheran church has stood at 400 S Logan Str...,A technicolor place of worship for Elevationis...,"Fifty-Two 80's,Cheesman Park,History Colorado ...","400 S Logan St Denver, Colorado United States",39.7091,-104.9823,"lewblank,Simon Jones,HighPriest,AngieSmangie,c...",2017-07-20 00:00:00,"Cheesman Park,Fifty-Two 80's,History Colorado ...","Blackfriars Friary,Santuario de Atotonilco,Her...",https://www.atlasobscura.com/places/internatio...,0.5337
523,House of Nicolas Flamel,"alchemy,occult,magic,restaurants,houses,homes",646,3042,Strange symbols adorn the architecture at 51 r...,The oldest stone house in Paris was built by i...,"Dans le Noir?,The Relic Crypt of St. Helena at...","51 rue de Montmorency Paris, 75003 France",48.8635,2.3531,"Allison,kez7585,Peccable,Traveling Maven,Scott...",2014-01-26 00:00:00,The Relic Crypt of St. Helena at Église Saint-...,"Boleskine House,Porta Alchemica,Faustův Dům (F...",https://www.atlasobscura.com/places/house-of-n...,0.506

## 4 

In [15]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

In [16]:
resquery.reset_index(inplace = True, drop=True)
city=[]
country=[]
for i in range(resquery.shape[0]):
    lat = resquery.placeAlt[i]
    lon = resquery.placeLong[i]
    location = geolocator.reverse(str(lat)+","+str(lon))
    address = location.raw['address']
    city.append(address.get('state', ''))
    country.append(address.get('country', ''))
    
    
#print(city,country)
resquery.insert(resquery.shape[1], "city/state", city)
resquery.insert(resquery.shape[1], "country", country)


,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL,scores,city/state,country
0,Dream House,"optical oddities,eccentric homes,music,outside...",765,6007,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","Aretha Franklin Subway Tributes,Farm.One,The '...","275 Church St New York, New York, 10013 United...",40.7185,-74.0048,"seanmattison,mbison,Gray,Sofy,erosika,Blindcol...",2009-09-22 00:00:00,"Aretha Franklin Subway Tributes,Farm.One,The '...","Callejon de Hamel,Casa Neverlandia,Casa de Azu...",https://www.atlasobscura.com/places/dream-house,1.0000,New York,United States
1,The Ruins of St. Dunstan-in-the-East,"gardens,world war ii,churches,plants,military,...",1884,5754,The church of St.Dunstan-in-the-East has survi...,One of the few remaining casualties of the Lon...,"Billingsgate Roman House and Baths,Pudding Lan...",St. Dunstan-in-the-East St. Dunstan's Hill Lon...,51.5097,-0.0827,"Luke J Spencer,Goksu,Kas,adamjerodandrews,Mons...",2015-03-12 00:00:00,"Billingsgate Roman House and Baths,Philpot Lan...","Kaiser Wilhelm Memorial Church,Ruins of Holyro...",https://www.atlasobscura.com/places/the-ruins-...,0.9579,England,United Kingdom
2,Roosevelt Island Smallpox Hospital Ruins,"smallpox,quarantine,abandoned hospitals,abando...",1709,5153,Few diseases have had a greater impact on the ...,A crumbling hospital from the 1850s on Rooseve...,"United Nations Delegates Lounge,Roosevelt Isla...","Roosevelt Island New York, New York, 10044 Uni...",40.7515,-73.9596,"Rebekah Otto,Sharon Blackstone,spavalice,mbiso...",2010-01-13 00:00:00,"Roosevelt Island Cat Sanctuary,Strecker Memori...","Ruins of Lazaretto de Isla de Cabras,Glenn Dal...",https://www.atlasobscura.com/places/roosevelt-...,0.8578,New York,United States
3,The Hardy Tree,"ao loves halloween 2022,gravestones,trees,chur...",859,4042,"Inside an ancient London churchyard, an ash tr...",This churchyard arbor is surrounded by hundred...,"Platform 9 3/4,Word on the Water,British Libra...","Pancras Road London, England, NW1 1UL United K...",51.5349,-0.1309,"sarahvjanet,Pengwn,SEANETTA,giloscope,Stevren,...",2015-07-03 00:00:00,"Word on the Water,Platform 9 3/4,British Libra...",Convento de la Santa Cruz (Convent of Santa Cr...,https://www.atlasobscura.com/places/the-hardy-...,0.6729,England,United Kingdom
4,Santa Maria della Concezione Crypts,"capuchins,marquis de sade,mark twain,ossuaries...",2070,3529,"In 1775, the Marquis de Sade wrote of it, “I h...",The crypts of Capuchin friars decorated with t...,"Saint Victoria's Incorruptible Body,Keats-Shel...","Via Vittorio Veneto, 27 Rome, 00187 Italy",41.9047,12.4882,"Dylan,mshkrvn,Annetta Black,The Dubious Hausfr...",2009-08-09 00:00:00,"Keats-Shelley Memorial House,Zuccari Palace,Sa...","The Czech's Capuchin Crypt,Santa Maria dell'Or...",https://www.atlasobscura.com/places/santa-mari...,0.5875,Lazio,Italia
5,International Church of Cannabis,"cannabis,marijuana,drugs,religion,churches,sac...",607,3206,A Lutheran church has stood at 400 S Logan Str...,A technicolor place of worship for Elevationis...,"Fifty-Two 80's,Cheesman Park,History Colorado ...","400 S Logan St Denver, Colorado United States",39.7091,-104.9823,"lewblank,Simon Jones,HighPriest,AngieSmangie,c...",2017-07-20 00:00:00,"Cheesman Park,Fifty-Two 80's,History Colorado ...","Blackfriars Friary,Santuario de Atotonilco,Her...",https://www.atlasobscura.com/places/internatio...,0.5337,Colorado,United States
6,House of Nicolas Flamel,"alchemy,occult,magic,restaurants,houses,homes",646,3042,Strange symbols adorn the architecture at 51 r...,The oldest stone house in Paris was built by i...,"Dans le Noir?,The Relic Crypt of St. Helena at...","51 rue de Montmorency Paris, 75003 France",48.8635,2.3531,"Allison,kez7585,Peccable,Traveling Maven,Scott...",2014-01-26 00:00:00,The Relic Crypt of St.

In [17]:
print(resquery.shape)

(10, 18)


In [23]:
import plotly.express as px
import plotly.graph_objs as go

fig = px.scatter_geo(resquery,lat="placeAlt", lon="placeLong",
                     size="numPeopleVisited", color="country",
                     projection="natural earth",
                     labels={'placeAlt':'latitude','placeLong':'longitude','placeAdress':'adress','numPeopleVisited':'number of people who visited this place'},
                     hover_data=["placeAdress","city/state"],hover_name="placeName")
fig.show()

#### command line

In [102]:
#Command Line question
forCommandLine = data[data["placeAdress"].str.contains("Italy")]
print("Total number of places in Italy:", len(forCommandLine))
print("The average of visitors in Italy:", int((forCommandLine["numPeopleVisited"].sum())))
print("Who wants to visit England (in total):", forCommandLine["numPeopleWant"].sum())
print()

forCommandLine2 = data[data["placeAdress"].str.contains("Spain")]
print("Total number of places in Spain:", len(forCommandLine2))
print("The average of visitors in Spain:", int((forCommandLine2["numPeopleVisited"].sum())/len(forCommandLine)))
print("Who wants to visit England (in total):", forCommandLine2["numPeopleWant"].sum())
print()


forCommandLine3 = data[data["placeAdress"].str.contains("France")]
print("Total number of places in France:", len(forCommandLine3))
print("The average of visitors in France:", int((forCommandLine3["numPeopleVisited"].sum())/len(forCommandLine)))
print("Who wants to visit France (in total):", forCommandLine3["numPeopleWant"].sum())
print()


forCommandLine4 = data[data["placeAdress"].str.contains("England")]
print("Total number of places in England:", len(forCommandLine4))
print("The average of visitors in England:", int((forCommandLine4["numPeopleVisited"].sum())/len(forCommandLine4)))
print("Who wants to visit England (in total):", forCommandLine4["numPeopleWant"].sum())
print()



forCommandLine5 = data[data["placeAdress"].str.contains("United States")]
print("Total number of places in the United States:", len(forCommandLine5))
print("The average of visitors in United States:", int((forCommandLine5["numPeopleVisited"].sum())/len(forCommandLine5)))
print("Who wants to visit United States (in total):", forCommandLine5["numPeopleWant"].sum())
print()

Total number of places in Italy: 182
The average of visitors in Italy: 68791
Who wants to visit England (in total): 152906

Total number of places in Spain: 82
The average of visitors in Spain: 212
Who wants to visit England (in total): 64587

Total number of places in France: 188
The average of visitors in France: 446
Who wants to visit France (in total): 189973

Total number of places in England: 367
The average of visitors in England: 476
Who wants to visit England (in total): 389820

Total number of places in the United States: 4234
The average of visitors in United States: 431
Who wants to visit United States (in total): 3966022



## 5 Bonus Question

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#TODO
#Gestisci il caso in cui non siano tutti e tre, come caloli il final score?
#gestisci il caso inc ui una parola non sia nel documento (=print(parola non presente in nessun documento))
#Ottimizzazione: carica i dizionari al posto di prenderli sempre

In [75]:
#The function cleaning takes as input a string (a description) and returns the cleaned description as a list of strings
data['cleanName'] = data.placeName.apply(lambda row: " ".join(cleaning(row)))

In [78]:
#The function cleaning takes as input a string (a description) and returns the cleaned description as a list of strings
data['cleanAdress'] = data.placeAdress.apply(lambda row: " ".join(cleaning(row)))

In [156]:
def query_function(data, nameCol,q):
    dic1 = createFirstDic(data, nameCol)
    dic2 = createSecondDic(data, dic1, nameCol)
    s = query(q,dic1,dic2)
    return s

In [157]:
def complex_query(data):
    boolDesc = False
    boolName = False
    boolAdress = False

    set_list = []
    q_list = []
    q1 = input('Insert the query that you want to find in the places descriptions: [if you dont want to search for anything just press Enter]')
    if q1 != "":
        boolDesc = True
        q1_clean = cleaning(q1)
        set_list.append(query_function(data, "cleanDesc", q1_clean))
        q_list.append(q1_clean)
    else:
        q_list.append("")

    q2 = input('Insert the query that you want to find in the Names: [if you dont want to search for anything just press Enter]')
    if q2 != "":
        boolName = True
        q2_clean = cleaning(q2)
        set_list.append(query_function(data, "cleanName", q2_clean))
        q_list.append(q2_clean)
    else:
        q_list.append("")


    q3 = input('Insert the query that you want to find in the Adress: [if you dont want to search for anything just press Enter]')
    if q3 != "":
        boolAdress = True
        q3_clean = cleaning(q3)
        set_list.append(query_function(data, "cleanAdress", q3_clean))
        q_list.append(q3_clean)
    else:
        q_list.append("")


    bool_arr = [boolDesc, boolName, boolAdress]
    return set_list, bool_arr, q_list

In [158]:
def find_df_qv(data,q,colName):
    vectorizer = TfidfVectorizer(use_idf=True, analyzer='word',sublinear_tf=True)
    x =vectorizer.fit_transform(data[colName]).todense()
    df = pd.DataFrame(x, columns = vectorizer.get_feature_names_out())
    qv = vectorizer.transform([" ".join(q)]).todense()
    return df, qv


In [159]:
def query_tfidf(data: pd.DataFrame) -> pd.DataFrame:
    
    set_list, bool_arr, q_list = complex_query(data)

    dic = {0: "cleanDesc", 1: "cleanName", 2: "cleanAdress"}

    #create an interesction of all the query
    s = set()
    if len(set_list) > 0:   
        s = set_list[0]
        for x in set_list:
            s = s.intersection(x)

    # res_query will be our new dataframe such that contains only the places with description that 
    # contains all the words in the query
    res_query = data[data['placeName'].isin(s)]

    for bool in range(len(bool_arr)):
        if bool_arr[bool] == True:
            df, qv = find_df_qv(data, q_list[bool], dic[bool])

            # since later we'll need to compute the norm of this vector, we create an np array with the tfidf of the query
            qv = np.array(qv[0,:])

            scores = cosine_score(res_query,df,qv)    
            res_query.insert(res_query.shape[1], f"CS_{dic[bool]}",scores ) 

    return res_query

In [168]:
prova = query_tfidf(data)

100%|██████████| 7200/7200 [00:00<00:00, 52610.47it/s]


In [169]:
prova

,Unnamed: 0,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,...,placeEditors,placePubDate,placeRelatedList,placeRelatedPlace,placeURL,cleanDesc,cleanName,cleanAdress,CS_cleanDesc,CS_cleanName
6,6,American Prohibition Museum,"speakeasies,prohibition,law,food museums,alcoh...",368,832,When the 18th Amendment to the U.S. Constituti...,"Performers, intricate sets, and a real speakea...","The Paris Market,Riverside Plant Hotel,The Mar...","209 W. St. Julian Street Savannah, Georgia Uni...",32.0806,...,"Edward Denny,Collector of Experiences,theparan...",2018-11-26 00:00:00,"The Paris Market,Riverside Plant Hotel,The Mar...","Booze History Museum,Linos tou Charilaou (Omod...",https://www.atlasobscura.com/places/american-p...,th amend u constitut outlaw alcohol deleteri d...,american prohibit museum,w st julian street savannah georgia unit state,0.081833,0.565670
146,146,American Classic Arcade Museum,"pinball,arcades,video games,games,museums and ...",468,1022,"Housed inside New Hampshire’s Funspot, which h...",The largest arcade museum in the world chronic...,"Endicott Rock,Archie Andrews Statue,Claude Rai...","579 Endicott Street North Laconia, New Hampshi...",43.6130,...,"thomasharper,tehutika,Mark Mones,e1savage",2014-04-04 00:00:00,"Endicott Rock,Archie Andrews Statue,Claude Rai...","Asheville Pinball Museum,Pinball Hall of Fame,...",https://www.atlasobscura.com/places/american-c...,hous insid new hampshir funspot hold guin worl...,american classic arcad museum,endicott street north laconia new hampshir u...,0.097970,0.483281
197,197,American Museum of Western Art,"brothels,americana,hotels,museums and collecti...",323,806,Housed in the old Navarre building across from...,The huge collection of paintings of the Americ...,"Mile-High Steps at the Colorado State Capitol,...","1727 Tremont Place Denver, Colorado United States",39.7447,...,"randibrown,bribolio,cyccommute",2017-11-27 00:00:00,"Denver Omelet Plaque,Big Blue Bear,Mile-High S...","Null Stern Hotel Museum,French Shore Interpret...",https://www.atlasobscura.com/places/american-m...,hous old navarr build across brown palac hotel...,american museum western art,tremont place denver colorado unit state,0.145389,0.525209
385,385,American Gothic House,"eccentric homes,shops,homes,architectural oddi...",380,784,Made famous in the 1930 painting by Grant Wood...,Home with Gothic windows featured in a famous ...,"What Cheer,Maharishi Vedic City,Richard Proenn...","300 Gothic St. Eldon, Iowa, 52554 United States",40.9211,...,"Facebook550112500,renkessler,smcripps04,drvand...",2011-06-29 00:00:00,"Maharishi Vedic City,What Cheer,Richard Proenn...","Krämerbrücke,Castello di Amorosa,Margalef,Hist...",https://www.atlasobscura.com/places/american-g...,made famou paint grant wood american gothic o...,american gothic hous,gothic st eldon iowa unit state,0.040367,0.571079
1750,1750,The American Geographical Society Library,"globes,maps,libraries",174,635,Within the campus of the University of Wiscons...,Literally a million fascinating cartographic a...,"Shorewood Ghost Train,North Point Water Tower,...","2311 E Hartford Ave Golda Meir Library, 3rd Fl...",43.0771,...,"Luke J Spencer,hrnick",2016-03-13 00:00:00,"Shorewood Ghost Train,North Point Water Tower,...","Royal Library of San Lorenzo de El Escorial,Ca...",https://www.atlasobscura.com/places/the-americ...,within campu univers wisconsin milwauke geogra...,american geograph societi librari,e hartford ave golda meir librari rd floor ea...,0.041972,0.450845
2042,2042,DANK Haus German American Cultural Center,"gallery,schools,immigration,culture,cities,mus...",278,613,DANK Haus Chicago has served as the city’s Ger...,"You can take a class, browse the library, or v...","Former Site of Selig Polyscope Film Studios,Me...","4740 N Western Ave Chicago, Illinois United St...",41.9680,...,Patti Swanson,2017-12-06 00:00:00,"Merz Apothecary,Former Site of Selig Polyscope...","Carl Weege Imm

In [160]:
def define_final_score(resComplexQuery):
    resComplexQuery["final_score"] = resComplexQuery.CS_cleanDesc* 0.15 + resComplexQuery.CS_cleanName * 0.50 + resComplexQuery.CS_cleanAdress* 0.35
    return resComplexQuery

## Filtering

### List of username

In [162]:
#This function return a pd.DataFrame with the filtering applied to the placeEditors
#To filter this right i create a regex string with all the names in the list
#The regex string will be equivalent to: (nameInTheList1 AND nameInTheList2 AND ... AND nameInTheListN)
#The regex for this operation is (?=.*Name1)(?=.*Name2)...(?=-*NameN)
def filterUsername(list: list, data: pd.DataFrame) -> pd.DataFrame:
    if list:
        return data
    else:
        filtered_data = data[data["placeEditors"].str.contains("(?=.*" + ")(?=.*".join(list) + ")")]
        return filtered_data

# Tags

In [164]:
def filterTags(list: list, data: pd.DataFrame) -> pd.DataFrame:
    if list:
        return data
    else:
        filtered_data = data[data["placeTags"].str.contains("(?=.*" + ")(?=.*".join(list) + ")")]
        return filtered_data

## Upper and lower bound

In [165]:

def filterNumPeople(upperbound = data["numPeopleVisited"].max(), lowerbound = 0) -> pd.DataFrame:
    filtered_data =  data.query(f'{lowerbound} < numPeopleVisited < {upperbound}')
    return filtered_data

## Final function

In [175]:
def filterFinalQuery(data: pd.DataFrame) -> pd.DataFrame :
    resComplexQuery = query_tfidf(data)
    resComplexQuery_fs = define_final_score(resComplexQuery)
    usernameList = input("Write the names of the users that worked on a page. [Separated by spaces]").split()
    tags = input("Write the names of the tags that you want to find in a page. [Separated by spaces]").split()
    upperBound = input("Write the maximum number of people that visited a certain location")
    lowerBound = input("Write the minimum number of people that visited a certain location")
    if not usernameList:
        resComplexQuery_fs = filterUsername(usernameList, resComplexQuery)
    if not tags:
        resComplexQuery_fs = filterTags(tags, resComplexQuery_fs)
    if upperBound and lowerBound:
        resComplexQuery_fs = filterNumPeople(int(upperBound), int(lowerBound))
    elif upperBound:
        resComplexQuery_fs = filterNumPeople(int(upperBound))
    elif (lowerBound):
        resComplexQuery_fs = filterNumPeople(lowerbound = int(lowerBound))
    return resComplexQuery_fs


    

    


In [176]:
#Try it out!!!
ex = filterFinalQuery(data)

100%|██████████| 7200/7200 [00:00<00:00, 21346.75it/s]
C:\Users\Matteo\AppData\Local\Temp\ipykernel_12572\3891487302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resComplexQuery["final_score"] = resComplexQuery.CS_cleanDesc* 0.15 + resComplexQuery.CS_cleanName * 0.50 + resComplexQuery.CS_cleanAdress* 0.35


In [177]:
ex.head()

,Unnamed: 0,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAdress,placeAlt,...,placeRelatedList,placeRelatedPlace,placeURL,cleanDesc,cleanName,cleanAdress,CS_cleanDesc,CS_cleanName,CS_cleanAdress,final_score
197,197,American Museum of Western Art,"brothels,americana,hotels,museums and collecti...",323,806,Housed in the old Navarre building across from...,The huge collection of paintings of the Americ...,"Mile-High Steps at the Colorado State Capitol,...","1727 Tremont Place Denver, Colorado United States",39.7447,...,"Denver Omelet Plaque,Big Blue Bear,Mile-High S...","Null Stern Hotel Museum,French Shore Interpret...",https://www.atlasobscura.com/places/american-m...,hous old navarr build across brown palac hotel...,american museum western art,tremont place denver colorado unit state,0.145389,0.525209,0.377556,0.416558
